**[CNN-01]** 必要なモジュールをインポートして、乱数のシードを設定します。

In [1]:
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt
from tensorflow.examples.tutorials.mnist import input_data

np.random.seed(20160703)
tf.set_random_seed(20160703)

**[CNN-02]** MNISTのデータセットを用意します。

In [2]:
mnist = input_data.read_data_sets("/tmp/data/", one_hot=True)

Extracting /tmp/data/train-images-idx3-ubyte.gz
Extracting /tmp/data/train-labels-idx1-ubyte.gz
Extracting /tmp/data/t10k-images-idx3-ubyte.gz
Extracting /tmp/data/t10k-labels-idx1-ubyte.gz


**[CNN-03]** 1段目の畳み込みフィルターとプーリング層を定義します。

In [3]:
num_filters1 = 32

x = tf.placeholder(tf.float32, [None, 784])
x_image = tf.reshape(x, [-1,28,28,1])

W_conv1 = tf.Variable(tf.truncated_normal([5,5,1,num_filters1],
                                          stddev=0.1))
h_conv1 = tf.nn.conv2d(x_image, W_conv1,
                       strides=[1,1,1,1], padding='SAME')

b_conv1 = tf.Variable(tf.constant(0.1, shape=[num_filters1]))
h_conv1_cutoff = tf.nn.relu(h_conv1 + b_conv1)

h_pool1 =tf.nn.max_pool(h_conv1_cutoff, ksize=[1,2,2,1],
                        strides=[1,2,2,1], padding='SAME')

**[CNN-04]** 2段目の畳み込みフィルターとプーリング層を定義します。

In [4]:
num_filters2 = 64

W_conv2 = tf.Variable(
            tf.truncated_normal([5,5,num_filters1,num_filters2],
                                stddev=0.1))
h_conv2 = tf.nn.conv2d(h_pool1, W_conv2,
                       strides=[1,1,1,1], padding='SAME')

b_conv2 = tf.Variable(tf.constant(0.1, shape=[num_filters2]))
h_conv2_cutoff = tf.nn.relu(h_conv2 + b_conv2)

h_pool2 =tf.nn.max_pool(h_conv2_cutoff, ksize=[1,2,2,1],
                        strides=[1,2,2,1], padding='SAME')

**[CNN-05]** 全結合層、ドロップアウト層、ソフトマックス関数を定義します。

In [5]:
h_pool2_flat = tf.reshape(h_pool2, [-1, 7*7*num_filters2])

num_units1 = 7*7*num_filters2
num_units2 = 1024

w2 = tf.Variable(tf.truncated_normal([num_units1, num_units2]))
b2 = tf.Variable(tf.constant(0.1, shape=[num_units2]))
hidden2 = tf.nn.relu(tf.matmul(h_pool2_flat, w2) + b2)

keep_prob = tf.placeholder(tf.float32)
hidden2_drop = tf.nn.dropout(hidden2, keep_prob)

w0 = tf.Variable(tf.zeros([num_units2, 10]))
b0 = tf.Variable(tf.zeros([10]))
p = tf.nn.softmax(tf.matmul(hidden2_drop, w0) + b0)

**[CNN-06]** 誤差関数 loss、トレーニングアルゴリズム train_step、正解率 accuracy を定義します。

In [6]:
t = tf.placeholder(tf.float32, [None, 10])
loss = -tf.reduce_sum(t * tf.log(p))
train_step = tf.train.AdamOptimizer(0.0001).minimize(loss)
correct_prediction = tf.equal(tf.argmax(p, 1), tf.argmax(t, 1))
accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))

**[CNN-07]** セッションを用意して、Variable を初期化します。

In [7]:
sess = tf.InteractiveSession()
sess.run(tf.initialize_all_variables())
saver = tf.train.Saver()

**[CNN-08]** パラメーターの最適化を25000回繰り返します。

最終的に、テストセットに対して約99%の正解率が得られます。

In [8]:
i = 0
for _ in range(25000):
    i += 1
    batch_xs, batch_ts = mnist.train.next_batch(50)
    sess.run(train_step,
             feed_dict={x:batch_xs, t:batch_ts, keep_prob:0.5})
    if i % 500 == 0:
        loss_val, acc_val = sess.run([loss, accuracy],
            feed_dict={x:mnist.test.images, t:mnist.test.labels,
                       keep_prob:1.0})
        print ('Step: %d, Loss: %f, Accuracy: %f'
               % (i, loss_val, acc_val))
        saver.save(sess, 'cnn_session', global_step=i)

Step: 500, Loss: 1679.738281, Accuracy: 0.949800
Step: 1000, Loss: 1056.643066, Accuracy: 0.968600
Step: 1500, Loss: 836.698303, Accuracy: 0.974500
Step: 2000, Loss: 753.148926, Accuracy: 0.974300
Step: 2500, Loss: 605.285156, Accuracy: 0.980900
Step: 3000, Loss: 584.634216, Accuracy: 0.980800
Step: 3500, Loss: 588.606079, Accuracy: 0.980000
Step: 4000, Loss: 493.769531, Accuracy: 0.984200
Step: 4500, Loss: 476.820618, Accuracy: 0.983600
Step: 5000, Loss: 437.336884, Accuracy: 0.984100
Step: 5500, Loss: 409.274231, Accuracy: 0.985700
Step: 6000, Loss: 399.363037, Accuracy: 0.987100
Step: 6500, Loss: 384.203064, Accuracy: 0.986900
Step: 7000, Loss: 390.318451, Accuracy: 0.985500
Step: 7500, Loss: 336.461456, Accuracy: 0.988300
Step: 8000, Loss: 329.810852, Accuracy: 0.988200
Step: 8500, Loss: 347.900208, Accuracy: 0.987700
Step: 9000, Loss: 363.413147, Accuracy: 0.986800
Step: 9500, Loss: 359.362152, Accuracy: 0.987700
Step: 10000, Loss: 341.048218, Accuracy: 0.988500
Step: 10500, Loss:

**[CNN-09]** セッション情報を保存したファイルが生成されていることを確認します。

In [9]:
!ls cnn_session*

cnn_session-23000	cnn_session-24000	cnn_session-25000
cnn_session-23000.meta	cnn_session-24000.meta	cnn_session-25000.meta
cnn_session-23500	cnn_session-24500
cnn_session-23500.meta	cnn_session-24500.meta
